In [1]:
from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)

from idaes.core.util import scaling as iscale

# Import plotting functions
import matplotlib.pyplot as plt

# Import numpy library 
import numpy as np

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

import idaes.logger as idaeslog

from pyomo.opt import TerminationCondition, SolverStatus

# Import the degrees_of_freedom function from the idaes.core.util.model_statistics package
# DOF = Number of Model Variables - Number of Model Constraints
from idaes.core.util.model_statistics import degrees_of_freedom

# Import the Generic Parameter Block
from idaes.models.properties.modular_properties.base.generic_property import (
        GenericParameterBlock)

# Import unit models from the model library
from idaes.models_extra.column_models.tray_column import TrayColumn
from idaes.models_extra.column_models.condenser \
    import CondenserType, TemperatureSpec

In [2]:
from HFCs_bmimPF6_PR_m import configuration

In [3]:
# Create the ConcreteModel and the FlowsheetBlock, and attach the flowsheet block to it.
m = ConcreteModel()

m.fs = FlowsheetBlock(default={"dynamic": False}) 

# Add properties parameter blocks to the flowsheet with specifications
m.fs.props = GenericParameterBlock(default=configuration)

In [4]:
m.fs.unit = TrayColumn(default={
                       "number_of_trays": 6,
                       "feed_tray_location": 4,
                       "condenser_type": CondenserType.partialCondenser,
                       "condenser_temperature_spec":
                           TemperatureSpec.customTemperature,
                       "property_package": m.fs.props,
                       "has_heat_transfer": False,
                       "has_pressure_change": False})

2022-08-01 08:59:45 [WARNING] idaes.core.base.unit_model: DEPRECATION: <class 'idaes.core.base.process_block._ScalarTrayColumn'> has overloaded the initialize method. In v2.0, IDAES Will be moving to having a centralized initialize method which calls unit-specific initialize_build methods instead. Model developers should update their models to implement the initialize_build method instead of overloading initialize.
2022-08-01 08:59:45 [WARNING] idaes.core.base.unit_model: DEPRECATION: <class 'idaes.models_extra.column_models.tray.TrayData'> has overloaded the initialize method. In v2.0, IDAES Will be moving to having a centralized initialize method which calls unit-specific initialize_build methods instead. Model developers should update their models to implement the initialize_build method instead of overloading initialize.
    (type=<class 'pyomo.core.base.var.IndexedVar'>) on block
    fs.unit.rectification_section[2] with a new Component (type=<class
    'pyomo.core.base.var.Indexe

In [5]:
m.fs.props.PR_kappa['R32', 'R125'].fix(0.009)
m.fs.props.PR_kappa['R125', 'R32'].fix(0.007)
m.fs.props.PR_kappa['R32', 'bmimPF6'].fix(-0.02608)
m.fs.props.PR_kappa['bmimPF6', 'R32'].fix(-0.0704)
m.fs.props.PR_kappa['bmimPF6', 'R125'].fix(0.349073)
m.fs.props.PR_kappa['R125', 'bmimPF6'].fix(0.058856)



In [6]:
# m.fs.unit.feed_entr.flow_mol_phase_comp[0, 'Liq',"R32"].fix(1e-5)
# m.fs.unit.feed_entr.flow_mol_phase_comp[0, 'Liq',"R125"].fix(1e-5)
# m.fs.unit.feed_entr.flow_mol_phase_comp[0, 'Liq',"bmimPF6"].fix(10)
# m.fs.unit.feed_entr.flow_mol_phase_comp[0, 'Vap',"R32"].fix(0.1)
# m.fs.unit.feed_entr.flow_mol_phase_comp[0, 'Vap',"R125"].fix(0.1)

# m.fs.unit.feed_entr.temperature.fix(298)
# m.fs.unit.feed_entr.pressure.fix(700000)
m.fs.unit.feed_entr.mole_frac_comp[0, "R32"].fix(0.3)
m.fs.unit.feed_entr.mole_frac_comp[0, "R125"].fix(0.3)
m.fs.unit.feed_entr.mole_frac_comp[0, "bmimPF6"].fix(0.4)

m.fs.unit.feed_entr.flow_mol.fix(2000)
m.fs.unit.feed_entr.temperature.fix(298)
m.fs.unit.feed_entr.pressure.fix(100000)

In [7]:
# m.fs.unit.feed_entr.flow_mol_phase_comp[0, 'Liq',"R32"].fix(1e-5)
# m.fs.unit.feed_entr.flow_mol_phase_comp[0, 'Liq',"R125"].fix(1e-5)
# m.fs.unit.feed_entr.flow_mol_phase_comp[0, 'Liq',"bmimPF6"].fix(0.5)
# m.fs.unit.feed_entr.flow_mol_phase_comp[0, 'Vap',"R32"].fix(0.4)
# m.fs.unit.feed_entr.flow_mol_phase_comp[0, 'Vap',"R125"].fix(0.4)
# m.fs.unit.feed.temperature.fix(298)
# m.fs.unit.feed.pressure.fix(500000)
m.fs.unit.feed.mole_frac_comp[0, "R32"].fix(0.4)
m.fs.unit.feed.mole_frac_comp[0, "R125"].fix(0.4)
m.fs.unit.feed.mole_frac_comp[0, "bmimPF6"].fix(0.2)
# m.fs.unit.feed.mole_frac_comp[0, "R32"].fix(0.3)
# m.fs.unit.feed.mole_frac_comp[0, "R125"].fix(0.3)
# m.fs.unit.feed.mole_frac_comp[0, "bmimPF6"].fix(0.4)

m.fs.unit.feed.flow_mol.fix(100)
m.fs.unit.feed.temperature.fix(298)
m.fs.unit.feed.pressure.fix(100000)

In [8]:
m.fs.unit.condenser.reflux_ratio.fix(0.25)

m.fs.unit.condenser.condenser_pressure.fix(100000)

m.fs.unit.reboiler.boilup_ratio.fix(1.0)

m.fs.unit.condenser.vapor_outlet.temperature.fix(286)

In [9]:
# Set scaling for model components

def scale_variables(m):
    for var in m.fs.component_data_objects(Var, descend_into=True):
        if 'flow_mol' in var.name:
            iscale.set_scaling_factor(var, 1)
        if 'temperature' in var.name:
            iscale.set_scaling_factor(var, 1e-2)
            
        if 'pressure' in var.name:
            iscale.set_scaling_factor(var, 1e-5)
        if 'enth_mol' in var.name:
            iscale.set_scaling_factor(var, 1e-4)
        if 'enth_mol_phase' in var.name:
            iscale.set_scaling_factor(var, 1e-4)
        if 'mole_frac' in var.name:
            iscale.set_scaling_factor(var, 1e1)
        if 'mole_frac_comp' in var.name:
            iscale.set_scaling_factor(var, 1e1)
        if 'entr_mol' in var.name:
            iscale.set_scaling_factor(var, 1e-4)
        if "_enthalpy_flow_term" in var.name:
            iscale.set_scaling_factor(var, 1e-4)
        if 'heat' in var.name:
            iscale.set_scaling_factor(var, 1e-15)
        if 'work' in var.name:
            iscale.set_scaling_factor(var, 1e-2)
        if 'flow_mol_phase_comp' in var.name:
            iscale.set_scaling_factor(var, 1e-2)
        if '_enthalpy_flow' in var.name:
            iscale.set_scaling_factor(var, 1e-4)
    
#     # manual scaling
    for part in ('condenser','reboiler','feed_entr_tray','feed_tray',
                 "rectification_section",'stripping_section'):
        block = getattr(m.fs.unit, part)
        if part == "rectification_section":
            for b in block:
                if hasattr(block[b], "properties_in_liq"):
                    iscale.set_scaling_factor(
                        block[b].properties_in_liq[0.0].mole_frac_comp, 1e1)
                    iscale.set_scaling_factor(
                        block[b].properties_in_liq[0.0].log_mole_frac_phase_comp, 1e-1)
                    iscale.set_scaling_factor(
                        block[b].properties_in_liq[0.0].enth_mol_phase['Liq'], 1e-4)
                    iscale.set_scaling_factor(
                        block[b].properties_in_liq[0.0].enth_mol_phase['Vap'], 1e-4)


                if hasattr(block[b], "properties_in_vap"):
                    iscale.set_scaling_factor(
                        block[b].properties_in_vap[0.0].mole_frac_comp, 1e1)
                    iscale.set_scaling_factor(
                        block[b].properties_in_vap[0.0].log_mole_frac_phase_comp, 1e-1)
                    iscale.set_scaling_factor(
                        block[b].properties_in_vap[0.0].enth_mol_phase['Liq'], 1e-4)
                    iscale.set_scaling_factor(
                        block[b].properties_in_vap[0.0].enth_mol_phase['Vap'], 1e-4)

                if hasattr(block[b], "properties_out"):
                    iscale.set_scaling_factor(
                        block[b].properties_out[0.0].mole_frac_comp, 1e1)
                    iscale.set_scaling_factor(
                        block[b].properties_out[0.0].log_mole_frac_phase_comp, 1e-1)
                    iscale.set_scaling_factor(
                        block[b].properties_out[0.0].enth_mol_phase['Liq'], 1e-4)
                    iscale.set_scaling_factor(
                        block[b].properties_out[0.0].enth_mol_phase['Vap'], 1e-4)

                iscale.set_scaling_factor(    
                        block[b].e_flow_liq_out, 1e-1)
                iscale.set_scaling_factor(    
                        block[b].e_flow_vap_out, 1e-1)
                iscale.set_scaling_factor(    
                        block[b].e_mole_frac_vap_out, 1e1)
                iscale.set_scaling_factor(    
                        block[b].e_mole_frac_liq_out, 1e1)
                
        elif part == "stripping_section":
            for b in block:
                if hasattr(block[b], "properties_in_liq"):
                    iscale.set_scaling_factor(
                        block[b].properties_in_liq[0.0].mole_frac_comp, 1e1)
                    iscale.set_scaling_factor(
                        block[b].properties_in_liq[0.0].log_mole_frac_phase_comp, 1e-1)
                    iscale.set_scaling_factor(
                        block[b].properties_in_liq[0.0].enth_mol_phase['Liq'], 1e-4)
                    iscale.set_scaling_factor(
                        block[b].properties_in_liq[0.0].enth_mol_phase['Vap'], 1e-4)


                if hasattr(block[b], "properties_in_vap"):
                    iscale.set_scaling_factor(
                        block[b].properties_in_vap[0.0].mole_frac_comp, 1e1)
                    iscale.set_scaling_factor(
                        block[b].properties_in_vap[0.0].log_mole_frac_phase_comp, 1e-1)
                    iscale.set_scaling_factor(
                        block[b].properties_in_vap[0.0].enth_mol_phase['Liq'], 1e-4)
                    iscale.set_scaling_factor(
                        block[b].properties_in_vap[0.0].enth_mol_phase['Vap'], 1e-4)

                if hasattr(block[b], "properties_out"):
                    iscale.set_scaling_factor(
                        block[b].properties_out[0.0].mole_frac_comp, 1e1)
                    iscale.set_scaling_factor(
                        block[b].properties_out[0.0].log_mole_frac_phase_comp, 1e-1)
                    iscale.set_scaling_factor(
                        block[b].properties_out[0.0].enth_mol_phase['Liq'], 1e-4)
                    iscale.set_scaling_factor(
                        block[b].properties_out[0.0].enth_mol_phase['Vap'], 1e-4)                    
                
                iscale.set_scaling_factor(    
                        block[b].e_flow_liq_out, 1e-1)
                iscale.set_scaling_factor(    
                        block[b].e_flow_vap_out, 1e-1)
                iscale.set_scaling_factor(    
                        block[b].e_mole_frac_vap_out, 1e1)
                iscale.set_scaling_factor(    
                        block[b].e_mole_frac_liq_out, 1e1)                    
        else: 
            if hasattr(block, "control_volume"):
                iscale.set_scaling_factor(
                    block.control_volume.properties_in[0.0].enth_mol_phase, 1e-4)
                iscale.set_scaling_factor(
                    block.control_volume.properties_out[0.0].enth_mol_phase, 1e-4)
                iscale.set_scaling_factor(
                    block.control_volume.properties_in[0.0].mole_frac_comp, 1e1)
                iscale.set_scaling_factor(
                    block.control_volume.properties_out[0.0].mole_frac_comp, 1e1)
                
                if hasattr(block.control_volume, "heat"):
                    iscale.set_scaling_factor(block.control_volume.heat, 1e-12)
                    
                if hasattr(block.control_volume, "work"):
                    iscale.set_scaling_factor(block.control_volume.work, 1e-3)
 
            if hasattr(block, "properties_in_liq"):
                    iscale.set_scaling_factor(
                        block.properties_in_liq[0.0].mole_frac_comp, 1e1)
                    iscale.set_scaling_factor(
                        block.properties_in_liq[0.0].log_mole_frac_phase_comp, 1e-1)
                    iscale.set_scaling_factor(
                        block.properties_in_liq[0.0].enth_mol_phase['Liq'], 1e-4)
                    iscale.set_scaling_factor(
                        block.properties_in_liq[0.0].enth_mol_phase['Vap'], 1e-4)

            if hasattr(block, "properties_in_vap"):
                iscale.set_scaling_factor(
                    block.properties_in_vap[0.0].mole_frac_comp, 1e1)
                iscale.set_scaling_factor(
                    block.properties_in_vap[0.0].log_mole_frac_phase_comp, 1e-1)
                iscale.set_scaling_factor(
                    block.properties_in_vap[0.0].enth_mol_phase['Liq'], 1e-4)
                iscale.set_scaling_factor(
                    block.properties_in_vap[0.0].enth_mol_phase['Vap'], 1e-4)

            if hasattr(block, "properties_in_feed_entr"):
                iscale.set_scaling_factor(
                    block.properties_in_feed_entr[0.0].mole_frac_comp, 1e1)
                iscale.set_scaling_factor(
                    block.properties_in_feed_entr[0.0].log_mole_frac_phase_comp, 1e-1)
                iscale.set_scaling_factor(
                    block.properties_in_feed_entr[0.0].enth_mol_phase['Liq'], 1e-4)
                iscale.set_scaling_factor(
                    block.properties_in_feed_entr[0.0].enth_mol_phase['Vap'], 1e-4)            

            if hasattr(block, "properties_in_feed"):
                iscale.set_scaling_factor(
                    block.properties_in_feed[0.0].mole_frac_comp, 1e1)
                iscale.set_scaling_factor(
                    block.properties_in_feed[0.0].log_mole_frac_phase_comp, 1e-1)
                iscale.set_scaling_factor(
                    block.properties_in_feed[0.0].enth_mol_phase['Liq'], 1e-4)
                iscale.set_scaling_factor(
                    block.properties_in_feed[0.0].enth_mol_phase['Vap'], 1e-4)

            if hasattr(block, "properties_out"):
                iscale.set_scaling_factor(
                    block.properties_out[0.0].mole_frac_comp, 1e1)
                iscale.set_scaling_factor(
                    block.properties_out[0.0].log_mole_frac_phase_comp, 1e-1)
                iscale.set_scaling_factor(
                    block.properties_out[0.0].enth_mol_phase['Liq'], 1e-4)
                iscale.set_scaling_factor(
                    block.properties_out[0.0].enth_mol_phase['Vap'], 1e-4)
            
            if part == 'feed_tray':
                iscale.set_scaling_factor(    
                        block.e_flow_liq_out, 1)
                iscale.set_scaling_factor(    
                        block.e_flow_vap_out, 1)
                iscale.set_scaling_factor(    
                        block.e_mole_frac_vap_out, 1e1)
                iscale.set_scaling_factor(    
                        block.e_mole_frac_liq_out, 1e1)
                
            if part == 'feed_entr_tray':
                iscale.set_scaling_factor(    
                        block.e_flow_liq_out, 1)
                iscale.set_scaling_factor(    
                        block.e_flow_vap_out, 1)
                iscale.set_scaling_factor(    
                        block.e_mole_frac_vap_out, 1e1)
                iscale.set_scaling_factor(    
                        block.e_mole_frac_liq_out, 1e1)
                
            if part == 'condenser':
                iscale.set_scaling_factor(    
                        block.e_liq_frac, 1e1)
                iscale.set_scaling_factor(    
                        block.e_reflux_flow, 1)            

            if part == 'reboiler':
                iscale.set_scaling_factor(    
                        block.e_vap_flow, 1)
                iscale.set_scaling_factor(    
                        block.e_vap_frac, 1e1) 
                
    iscale.calculate_scaling_factors(m)
    
def scale_constraints(m):
    # set scaling for unit constraints
    for part in ('condenser','reboiler','feed_entr_tray','feed_tray','rectification_section','stripping_section'):
        block = getattr(m.fs.unit, part)
        
        # flash constraints
        if hasattr(block, 'material_splitting_eqn'):
            for (t, o, j), c in block.material_splitting_eqn.items():
                iscale.constraint_scaling_transform(c, 1e1, overwrite=False)
        if hasattr(block, 'temperature_equality_eqn'):
            for (t, o), c in block.temperature_equality_eqn.items():
                iscale.constraint_scaling_transform(c, 1e1, overwrite=False)
        if hasattr(block, 'molar_enthalpy_equality_eqn'):
            for (t, o), c in block.molar_enthalpy_equality_eqn.items():
                iscale.constraint_scaling_transform(c, 1e-4, overwrite=False)
        if hasattr(block, 'molar_enthalpy_splitting_eqn'):
            for (t, o), c in block.molar_enthalpy_splitting_eqn.items():
                iscale.constraint_scaling_transform(c, 1e-4, overwrite=False)
        if hasattr(block, 'pressure_equality_eqn'):
            for (t, o), c in block.pressure_equality_eqn.items():
                iscale.constraint_scaling_transform(c, 1e-5, overwrite=False)
        if hasattr(block, 'sum_split_frac'):
            for t, c in block.sum_split_frac.items():
                iscale.constraint_scaling_transform(c, 1e1, overwrite=False)
        if hasattr(block, 'split_fraction_eq'):
            for (t, o), c in block.split_fraction_eq.items():
                iscale.constraint_scaling_transform(c, 1e1, overwrite=False)
        if hasattr(block, 'enthalpy_mixing_equations'):
            for t, c in block.enthalpy_mixing_equations.items():
                iscale.constraint_scaling_transform(c, 1e-4, overwrite=False)   
        

        # heater and reactor only add 0D control volume constraints
        if hasattr(block, 'material_holdup_calculation'):
            for (t, p, j), c in block.material_holdup_calculation.items():
                iscale.constraint_scaling_transform(c, 1e-2, overwrite=False)
        if hasattr(block, 'material_balances'):
            for (t, p, j), c in block.material_balances.items():
                iscale.constraint_scaling_transform(c, 1e-2, overwrite=False)
        if hasattr(block, 'element_balances'):
            for (t, e), c in block.element_balances.items():
                iscale.constraint_scaling_transform(c, 1, overwrite=False)
        if hasattr(block, 'elemental_holdup_calculation'):
            for (t, e), c in block.elemental_holdup_calculation.items():
                iscale.constraint_scaling_transform(c, 1, overwrite=False)
        if hasattr(block, 'enthalpy_balances'):
            for t, c in block.enthalpy_balances.items():
                iscale.constraint_scaling_transform(c, 1e-4, overwrite=False)
        if hasattr(block, 'energy_holdup_calculation'):
            for (t, p), c in block.energy_holdup_calculation.items():
                iscale.constraint_scaling_transform(c, 1e-4, overwrite=False)
        if hasattr(block, 'pressure_balance'):
            for t, c in block.pressure_balance.items():
                iscale.constraint_scaling_transform(c, 1e-5, overwrite=False)
        if hasattr(block, 'sum_of_phase_fractions'):
            for t, c in block.sum_of_phase_fractions.items():
                iscale.constraint_scaling_transform(c, 1e1, overwrite=False)
        if hasattr(block, "material_accumulation_disc_eq"):
            for (t, p, j), c in block.material_accumulation_disc_eq.items():
                iscale.constraint_scaling_transform(c, 1e3, overwrite=False)
        if hasattr(block, "_teq_constraint_Vap_Liq"):
            for t, c in block._teq_constraint_Vap_Liq.items():
                iscale.constraint_scaling_transform(c, 1e1, overwrite=False)

    iscale.calculate_scaling_factors(m)

In [10]:
scale_variables(m)
scale_constraints(m)

In [11]:
print(degrees_of_freedom(m))

0


In [12]:
m.fs.unit.initialize() 

2022-08-01 08:59:56 [INFO] idaes.init.fs.unit: Begin initialization.
2022-08-01 08:59:56 [INFO] idaes.init.fs.unit.feed_tray: Begin initialization.
2022-08-01 08:59:56 [INFO] idaes.init.fs.unit.feed_tray.properties_in_feed: Starting initialization
2022-08-01 08:59:59 [INFO] idaes.init.fs.unit.feed_tray.properties_in_feed: Dew and bubble point initialization: optimal - Optimal Solution Found.
2022-08-01 08:59:59 [INFO] idaes.init.fs.unit.feed_tray.properties_in_feed: Equilibrium temperature initialization completed.
2022-08-01 09:00:06 [INFO] idaes.init.fs.unit.feed_tray.properties_in_feed: Phase equilibrium initialization: optimal - Optimal Solution Found.
2022-08-01 09:00:10 [INFO] idaes.init.fs.unit.feed_tray.properties_in_feed: Property initialization: optimal - Optimal Solution Found.
2022-08-01 09:00:10 [INFO] idaes.init.fs.unit.feed_tray.properties_in_liq: Starting initialization
2022-08-01 09:00:12 [INFO] idaes.init.fs.unit.feed_tray.properties_in_liq: Dew and bubble point initi

2022-08-01 09:02:32 [INFO] idaes.init.fs.unit.condenser.control_volume.properties_in: Phase equilibrium initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2022-08-01 09:02:36 [INFO] idaes.init.fs.unit.condenser.control_volume.properties_in: Property initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2022-08-01 09:02:36 [INFO] idaes.init.fs.unit.condenser.control_volume.properties_out: Starting initialization
WARNING (W1002): Setting Var
    'fs.unit.condenser.control_volume.properties_out[0.0].mole_frac_comp[bmimP
    F6]' to a numeric value `0.0` outside the bounds (1e-20, 1.001).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
2022-08-01 09:02:38 [INFO] idaes.init.fs.unit.condenser.control_volume.properties_out: Dew and bubble point initialization: optimal - Optimal Solution Found.
2022-08-01 09:02:38 [INFO] idaes.init.fs.unit.condenser.control_volume.properties_out: Eq

2022-08-01 09:07:46 [INFO] idaes.init.fs.unit.rectification_section[2]: Initialization complete, status infeasible - Converged to a locally infeasible point. Problem may be infeasible..
WARNING (W1002): Setting Var 'fs.unit.rectification_section[3].properties_in_v
    ap[0.0].mole_frac_comp[bmimPF6]' to a numeric value `0.0` outside the
    bounds (1e-20, 1.001).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
2022-08-01 09:07:46 [INFO] idaes.init.fs.unit.rectification_section[3]: Begin initialization.
2022-08-01 09:07:46 [INFO] idaes.init.fs.unit.rectification_section[3].properties_in_liq: Starting initialization
2022-08-01 09:07:48 [INFO] idaes.init.fs.unit.rectification_section[3].properties_in_liq: Dew and bubble point initialization: optimal - Optimal Solution Found.
2022-08-01 09:07:48 [INFO] idaes.init.fs.unit.rectification_section[3].properties_in_liq: Equilibrium temperature initialization completed.
2022-08-01 09:07:52 [INFO] idaes.init.fs.unit.rectifica

2022-08-01 09:09:15 [INFO] idaes.init.fs.unit.stripping_section[5]: Mass balance solve optimal - Optimal Solution Found.
2022-08-01 09:09:23 [INFO] idaes.init.fs.unit.stripping_section[5]: Mass and energy balance solve optimal - Optimal Solution Found.
2022-08-01 09:09:33 [INFO] idaes.init.fs.unit.stripping_section[5]: Mass, energy and pressure balance solve optimal - Optimal Solution Found.
2022-08-01 09:09:33 [INFO] idaes.init.fs.unit.stripping_section[5]: Initialization complete, status optimal - Optimal Solution Found.
WARNING (W1002): Setting Var
    'fs.unit.stripping_section[6].properties_in_vap[0.0].mole_frac_comp[bmimPF
    6]' to a numeric value `0.0` outside the bounds (1e-20, 1.001).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
2022-08-01 09:09:33 [INFO] idaes.init.fs.unit.stripping_section[6]: Begin initialization.
2022-08-01 09:09:33 [INFO] idaes.init.fs.unit.stripping_section[6].properties_in_liq: Starting initialization
2022-08-01 09:09:36 [INFO

In [13]:
# m.fs.unit.condenser.vapor_outlet.temperature.unfix()

In [14]:
solver = SolverFactory('ipopt')
# solver.options['bound_push'] = 1e-06
# solver.options['mu_init'] = 1e-06
solver.options = {'tol': 1e-6,'bound_push': 1e-4,
                 'max_iter':1000}

In [15]:
# m.fs.unit.feed.mole_frac_comp[0, "R32"].fix(0.45)
# m.fs.unit.feed.mole_frac_comp[0, "R125"].fix(0.45)
# m.fs.unit.feed.mole_frac_comp[0, "bmimPF6"].fix(0.1)

In [16]:
solve_status = solver.solve(m, tee=True)

Ipopt 3.13.2: tol=1e-06
bound_push=0.0001
max_iter=1000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale s

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  70  0.0000000e+00 3.98e+09 9.39e+07  -1.7 2.49e+01   2.1 9.23e-02 9.23e-02s 20
  71  0.0000000e+00 3.96e+09 9.34e+07  -1.7 1.02e+01   2.5 5.00e-03 5.00e-03s 20
  72  0.0000000e+00 3.95e+09 9.32e+07  -1.7 2.89e+01   2.1 2.10e-03 2.10e-03s 20
  73  0.0000000e+00 3.89e+09 9.18e+07  -1.7 1.15e+01   2.5 1.50e-02 1.50e-02s 20
  74  0.0000000e+00 3.72e+09 8.76e+07  -1.7 4.43e+00   2.9 4.52e-02 4.52e-02s 20
  75  0.0000000e+00 3.60e+09 2.74e+06  -1.7 1.33e+01   2.4 1.00e+00 1.00e+00s 20
  76  0.0000000e+00 2.37e+09 1.83e+06  -1.7 9.41e-01   3.8 3.41e-01 3.41e-01s 20
  77  0.0000000e+00 8.48e+08 9.10e+05  -1.7 3.15e+00   3.3 6.51e-01 6.51e-01s 20
  78r 0.0000000e+00 8.48e+08 1.00e+03   0.9 0.00e+00   2.8 0.00e+00 0.00e+00R  1
  79r 0.0000000e+00 1.15e+08 1.38e+04   0.9 8.47e+03    -  3.43e-03 9.91e-04f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  80r 0.0000000e+00 1.21e+07

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 170r 0.0000000e+00 6.31e+07 1.30e+03  -2.8 9.38e+01  -2.6 2.65e-03 3.58e-05h  9
 171r 0.0000000e+00 6.32e+07 1.25e+03  -2.8 4.57e+02  -3.1 4.88e-04 4.88e-04s 32
 172r 0.0000000e+00 6.64e+07 1.21e+03  -2.8 7.60e+03    -  1.01e-02 1.01e-02s 32
 173r 0.0000000e+00 8.95e+07 1.19e+03  -2.8 7.53e+03    -  2.15e-02 2.15e-02s 32
 174r 0.0000000e+00 2.24e+08 1.13e+03  -2.8 7.37e+03    -  5.21e-02 5.21e-02s 32
 175r 0.0000000e+00 5.36e+08 1.03e+03  -2.8 6.99e+03    -  8.51e-02 8.51e-02s 32
 176r 0.0000000e+00 1.55e+09 6.74e+02  -2.8 6.39e+03    -  3.42e-01 3.42e-01s 32
 177r 0.0000000e+00 1.51e+09 5.20e+02  -2.8 4.20e+03    -  2.28e-01 2.28e-01s 32
 178r 0.0000000e+00 1.03e+09 2.69e+02  -2.8 3.24e+03    -  4.83e-01 4.83e-01s 32
 179r 0.0000000e+00 4.36e+08 8.91e+01  -2.8 1.68e+03    -  6.72e-01 6.72e-01s 32
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 180r 0.0000000e+00 1.71e+07

In [17]:
# m.display()

In [18]:
from idaes.core.util.model_statistics import large_residuals_set

In [19]:
print(large_residuals_set(m))

ComponentSet(['fs.unit.rectification_section[2].material_mixing_equations[0.0,R125] (id=2434518780032)', 'fs.unit.rectification_section[2].enthalpy_mixing_equations[0.0] (id=2434518780128)', 'fs.unit.rectification_section[2].properties_in_vap[0.0]._t1_constraint_Vap_Liq (id=2434339148784)', 'fs.unit.rectification_section[2].properties_in_vap[0.0]._teq_constraint_Vap_Liq (id=2434339149008)', 'fs.unit.rectification_section[3].properties_in_liq[0.0].total_flow_balance (id=2434339156736)', 'fs.unit.rectification_section[3].properties_in_liq[0.0].component_flow_balances[R32] (id=2434518781088)', 'fs.unit.rectification_section[3].properties_in_liq[0.0].component_flow_balances[R125] (id=2434518781184)', 'fs.unit.rectification_section[3].properties_in_liq[0.0].phase_fraction_constraint[Vap] (id=2434518781376)', 'fs.unit.rectification_section[3].properties_in_liq[0.0]._t1_constraint_Vap_Liq (id=2434339157968)', 'fs.unit.rectification_section[3].properties_in_liq[0.0]._teq_constraint_Vap_Liq (id

In [20]:
print(degrees_of_freedom(m))

0


In [21]:
m.display()

Model unknown

  Variables:
    None

  Objectives:
    None

  Constraints:
    None

  Blocks:
    Block fs
    
      Variables:
        None
    
      Objectives:
        None
    
      Constraints:
        None
    
      Blocks:
        Block fs.props
        
          Variables:
            PR_kappa : PR binary interaction parameters
                Size=9, Index=fs.props.PR_kappa_index
                Key                    : Lower : Value    : Upper : Fixed : Stale : Domain
                      ('R125', 'R125') :  None :      0.0 :  None :  True :  True :  Reals
                       ('R125', 'R32') :  None :    0.007 :  None :  True :  True :  Reals
                   ('R125', 'bmimPF6') :  None : 0.058856 :  None :  True :  True :  Reals
                       ('R32', 'R125') :  None :    0.009 :  None :  True :  True :  Reals
                        ('R32', 'R32') :  None :      0.0 :  None :  True :  True :  Reals
                    ('R32', 'bmimPF6') :  None : -0.02

                    Key : Lower : Body          : Upper
                    0.0 :   0.0 : -925403.28125 :   0.0
                pressure_drop_equation : Size=1
                    Key : Lower : Body : Upper
                    0.0 :   0.0 :  0.0 :   0.0
            
              Blocks:
                Block fs.unit.rectification_section[2].properties_in_liq[0.0]
                
                  Variables:
                    flow_mol :  Total molar flowrate
                        Size=1, Index=None, Units=mol/s
                        Key  : Lower : Value             : Upper : Fixed : Stale : Domain
                        None : 1e-08 : 88.30531155488734 : 10000 : False : False : NonNegativeReals
                    mole_frac_comp : Mixture mole fractions
                        Size=3, Index=fs.props.component_list
                        Key     : Lower : Value                : Upper : Fixed : Stale : Domain
                           R125 : 1e-20 :  0.03727501499437618 : 1.001

                        Key                    : Lower : Body                   : Upper
                        ('Vap', 'Liq', 'R125') :   0.0 : 1.7313928069029316e-13 :   0.0
                         ('Vap', 'Liq', 'R32') :   0.0 : 1.6142642778049776e-13 :   0.0
                    log_mole_frac_phase_comp_eqn : Size=5
                        Key                : Lower : Body                   : Upper
                           ('Liq', 'R125') :   0.0 :  5.551115123125783e-16 :   0.0
                            ('Liq', 'R32') :   0.0 :  6.938893903907228e-17 :   0.0
                        ('Liq', 'bmimPF6') :   0.0 : -2.220446049250313e-15 :   0.0
                           ('Vap', 'R125') :   0.0 : 1.1102230246251565e-15 :   0.0
                            ('Vap', 'R32') :   0.0 : 2.7755575615628914e-16 :   0.0
                Block fs.unit.rectification_section[2].properties_in_vap[0.0]
                
                  Variables:
                    flow_mol :  Total molar flowra

                        Key                       : Lower : Body                    : Upper
                           ('Vap', 'Liq', 'R125') :   0.0 : -1.4617501653546583e-11 :   0.0
                            ('Vap', 'Liq', 'R32') :   0.0 :  -7.431877335761783e-11 :   0.0
                        ('Vap', 'Liq', 'bmimPF6') :   0.0 :                     0.0 :   0.0
                    log_mole_frac_comp_eqn : Size=3
                        Key     : Lower : Body                   : Upper
                           R125 :   0.0 : -3.703704010149522e-12 :   0.0
                            R32 :   0.0 : -5.643679967803905e-12 :   0.0
                        bmimPF6 :   0.0 : -9.870358708140569e-20 :   0.0
                    log_mole_frac_tbub_eqn : Size=2
                        Key                    : Lower : Body                   : Upper
                        ('Vap', 'Liq', 'R125') :   0.0 : 4.0374481535820905e-11 :   0.0
                         ('Vap', 'Liq', 'R32') :   0.0 :   4

                        Key                       : Lower : Body                    : Upper
                           ('Vap', 'Liq', 'R125') :   0.0 : -3.2289143581110125e-11 :   0.0
                            ('Vap', 'Liq', 'R32') :   0.0 :  -3.915978652457852e-12 :   0.0
                        ('Vap', 'Liq', 'bmimPF6') :   0.0 :   3.314050595098878e-12 :   0.0
                    log_mole_frac_comp_eqn : Size=3
                        Key     : Lower : Body                  : Upper
                           R125 :   0.0 : 3.425038030968608e-12 :   0.0
                            R32 :   0.0 : 5.042910533603617e-12 :   0.0
                        bmimPF6 :   0.0 : 1.734723475976807e-17 :   0.0
                    log_mole_frac_tbub_eqn : Size=2
                        Key                    : Lower : Body                   : Upper
                        ('Vap', 'Liq', 'R125') :   0.0 : -3.659084146789837e-11 :   0.0
                         ('Vap', 'Liq', 'R32') :   0.0 : -3.6593

...

                        Key                       : Lower : Body                    : Upper
                           ('Vap', 'Liq', 'R125') :   0.0 : -2.9394764400336726e-11 :   0.0
                            ('Vap', 'Liq', 'R32') :   0.0 : -1.0121015137087852e-11 :   0.0
                        ('Vap', 'Liq', 'bmimPF6') :   0.0 :  3.0800551442753844e-12 :   0.0
                    log_mole_frac_comp_eqn : Size=3
                        Key     : Lower : Body                   : Upper
                           R125 :   0.0 :  3.987921104453562e-12 :   0.0
                            R32 :   0.0 :  4.423683641618936e-12 :   0.0
                        bmimPF6 :   0.0 : -6.938893903907228e-17 :   0.0
                    log_mole_frac_tbub_eqn : Size=2
                        Key                    : Lower : Body                    : Upper
                        ('Vap', 'Liq', 'R125') :   0.0 :  -4.068456682659871e-11 :   0.0
                         ('Vap', 'Liq', 'R32') :   0.0 : -

                            Key                    : Lower : Body                   : Upper
                            ('Vap', 'Liq', 'R125') :   0.0 : -2.965128143017637e-11 :   0.0
                             ('Vap', 'Liq', 'R32') :   0.0 : -9.991341087811634e-12 :   0.0
                        log_mole_frac_phase_comp_eqn : Size=5
                            Key                : Lower : Body                    : Upper
                               ('Liq', 'R125') :   0.0 :  -3.751443600208404e-12 :   0.0
                                ('Liq', 'R32') :   0.0 : -4.3529069237990825e-12 :   0.0
                            ('Liq', 'bmimPF6') :   0.0 :     -0.4166666624560389 :   0.0
                               ('Vap', 'R125') :   0.0 :   3.732569808789776e-12 :   0.0
                                ('Vap', 'R32') :   0.0 :   3.735345366351339e-12 :   0.0
                    Block fs.unit.condenser.control_volume.properties_out[0.0]
                    
                      Variab

                            Key                    : Lower : Body                   : Upper
                            ('Vap', 'Liq', 'R125') :   0.0 : -4.075473292175502e-11 :   0.0
                             ('Vap', 'Liq', 'R32') :   0.0 : -8.217870828275409e-13 :   0.0
                        log_mole_frac_phase_comp_eqn : Size=5
                            Key                : Lower : Body                    : Upper
                               ('Liq', 'R125') :   0.0 : -1.3063439219251904e-11 :   0.0
                                ('Liq', 'R32') :   0.0 :  -3.069766663088558e-13 :   0.0
                            ('Liq', 'bmimPF6') :   0.0 :  -5.551115123125783e-16 :   0.0
                               ('Vap', 'R125') :   0.0 :   4.946043574705072e-12 :   0.0
                                ('Vap', 'R32') :   0.0 :  4.3604009292153023e-13 :   0.0
            Block fs.unit.reboiler
            
              Variables:
                boilup_ratio : Boilup ratio for reboile

                            Key                       : Lower : Body                   : Upper
                               ('Vap', 'Liq', 'R125') :   0.0 :    -9.753309271332e-14 :   0.0
                                ('Vap', 'Liq', 'R32') :   0.0 :  -8.08242361927114e-14 :   0.0
                            ('Vap', 'Liq', 'bmimPF6') :   0.0 : 1.6526559282973827e-12 :   0.0
                        log_mole_frac_comp_eqn : Size=3
                            Key     : Lower : Body                    : Upper
                               R125 :   0.0 :                     0.0 :   0.0
                                R32 :   0.0 : -2.7755575615628914e-16 :   0.0
                            bmimPF6 :   0.0 :  -6.938893903907228e-17 :   0.0
                        log_mole_frac_tbub_eqn : Size=2
                            Key                    : Lower : Body                   : Upper
                            ('Vap', 'Liq', 'R125') :   0.0 : 2.7755575615628914e-15 :   0.0
            

                            Key                       : Lower : Body                   : Upper
                               ('Vap', 'Liq', 'R125') :   0.0 :   2.95327096111464e-11 :   0.0
                                ('Vap', 'Liq', 'R32') :   0.0 :  1.017563810989941e-11 :   0.0
                            ('Vap', 'Liq', 'bmimPF6') :   0.0 : 2.1973580382195314e-13 :   0.0
                        log_mole_frac_comp_eqn : Size=3
                            Key     : Lower : Body                   : Upper
                               R125 :   0.0 :  -4.00457444982294e-12 :   0.0
                                R32 :   0.0 : -4.443390100306033e-12 :   0.0
                            bmimPF6 :   0.0 : -6.938893903907228e-17 :   0.0
                        log_mole_frac_tbub_eqn : Size=2
                            Key                    : Lower : Body                  : Upper
                            ('Vap', 'Liq', 'R125') :   0.0 : 4.088251959188938e-11 :   0.0
                  

In [22]:
# m.fs.unit.condenser.display()

In [23]:
# m.fs.unit.reboiler.display()

In [24]:
# m.fs.unit.display()

In [25]:
# m.fs.unit.rectification_section[2].display()
